In [2]:
import sys
try:
    sys.path.insert(0, "/usr/lib/python3.7/site-packages")
except FileNotFoundError:
    pass

import pandas as pd
import numpy as nm
import sys

import ufal.udpipe as ud
from ufal.udpipe import Model, Pipeline, ProcessingError
import pyconll, pyconll.util
from ufal.udpipe import Model, Pipeline
import os
import re
import wget

In [3]:
subj  = {"nsubj", "nsubj:pass"}
obj   = {"iobj", "obj"}
names = {"PRON", "PROPN", "NOUN"}

In [4]:
class nice_out:
    def __init__(self, L):
        self.L = L
        self.counter = 0
        self.per = 0
    def update(self):
        if(self.per == (self.counter*1000)//self.L):
            print("\r" + str(self.per/10) + "%" + " " + "#"*(self.per//20) + "_"*(50 - self.per//20), end = "")
            self.per += 1
        self.counter += 1
    def clear(self):
        print("\r" + " "*100, end = "\r")

## Process with UDPIPE

In [5]:
def process(text):
    processed = []
    text = text.split("\n")[1:]
    for word in text[1:]:
        word = word.split("\t")
        if(len(word) == 10):
            processed += [word[:4] + word[6:8]]
    return processed


def tag_ud(text, modelfile='udpipe_syntagrus.model'):
    udpipe_model_url = 'https://rusvectores.org/static/models/udpipe_syntagrus.model'
    udpipe_filename = udpipe_model_url.split('/')[-1]
    if(type(text) == str):
        text = [text]
        
    if not os.path.isfile(modelfile):
        print('UDPipe model not found. Downloading...', file=sys.stderr)
        wget.download(udpipe_model_url)
    
    model = Model.load(modelfile)
    process_pipeline = Pipeline(model, 'tokenize', Pipeline.DEFAULT, Pipeline.DEFAULT, 'conllu')
    
    items = []
    chk = nice_out(len(text))
    for line in text:
        chk.update()
        output = process_pipeline.process(line)
        output = output.split("# sent_id = ")
        for sentence in output[1:]:
            items += [process(sentence)]
    chk.clear()
    return items

## Change for PersonX

In [7]:
#def clear(item):
def get_root(item):
    for word in item:
        if(word[4] == '0'):
            return word[2]
    return "Нет"

def place_persons(item):
    subj_parent = "-1"
    subject = ""
    for word in item:
        if((word[5] in subj) and (word[3] in names)):
            if(subj_parent == "-1"):
                subject = word[2]
                word[1] = "ЧеловекX"
                word[2] = "ЧеловекX"
                subj_parent = word[4]
            else:
                if(word[2] == subject):
                    word[1] = "ЧеловекX"
                    word[2] = "ЧеловекX"

    if(subj_parent != "-1"):
        for word in item:
            if((word[5] in obj) and (word[3] in names) and word[4] == subj_parent):
                word[1] = "ЧеловекY"
                word[2] = "ЧеловекY"

def sout(item):
    out = ""
    for word in item:
        if(word[5] != "punct" and word[0] != 1):
            out += " "
        if(word[1] != None):
            out += word[1]
    return out

In [9]:
udpipe_url = "https://rusvectores.org/static/models/udpipe_syntagrus.model"
modelfile = "udpipe_syntagrus.model"
#textfile = wget.download(text_url)

text = ["Яндекс оставил Гугл далеко позади. Вася поднял мяч, кторый он когда-то оставил на чердаке",
       "Мастер увидел Маргариту с жёлтыми тюльпанами. Доброго Диму эксплуатирует злой Петя",
       "Скалозуб хвастается своими служебными успехами. Репетилов верит слуху. Андрей рос в селе. мальчишки гостю подчиняются.",
       "Она взяла его на руки оказались.",
       "Росс вспоминает о Кэрол. Джо благодаря своему новому агенту получает роль в новом фильме. Джоуи флиртуют с работницей."]

source = tag_ud(text=text, modelfile=modelfile)
for i in range(len(source)):
    print(str(i) + ") ", end = "")
    print(sout(source[i]), end = ""); print(" -> ", end = "")
    place_persons(source[i])
    print(sout(source[i]), end = ""); print("\n")

0)  Яндекс оставил Гугл далеко позади. ->  ЧеловекX оставил Гугл далеко позади.

1)  Вася поднял мяч, кторый он когда-то оставил на чердаке ->  ЧеловекX поднял ЧеловекY, кторый он когда-то оставил на чердаке

2)  Мастер увидел Маргариту с жёлтыми тюльпанами. ->  ЧеловекX увидел ЧеловекY с жёлтыми тюльпанами.

3)  Доброго Диму эксплуатирует злой Петя ->  Доброго ЧеловекY эксплуатирует злой ЧеловекX

4)  Скалозуб хвастается своими служебными успехами. ->  ЧеловекX хвастается своими служебными успехами.

5)  Репетилов верит слуху. ->  ЧеловекX верит ЧеловекY.

6)  Андрей рос в селе. ->  ЧеловекX рос в селе.

7)  мальчишки гостю подчиняются. ->  ЧеловекX ЧеловекY подчиняются.

8)  Она взяла его на руки оказались. ->  ЧеловекX взяла ЧеловекY на руки оказались.

9)  Росс вспоминает о Кэрол. ->  ЧеловекX вспоминает о Кэрол.

10)  Джо благодаря своему новому агенту получает роль в новом фильме. ->  ЧеловекX благодаря своему новому агенту получает ЧеловекY в новом фильме.

11)  Джоуи флиртуют с

## check sentence

In [10]:
text1 = "Остап медленно опустил свою руку в изрядно повреждённый стул"
modelfile='udpipe_syntagrus.model'
model = Model.load(modelfile)
process_pipeline = Pipeline(model, 'tokenize', Pipeline.DEFAULT, Pipeline.DEFAULT, 'conllu')
output = process_pipeline.process(text1)
tmp = process(output)
#place_persons(tmp)
#print(sout(tmp))
for word in tmp:
    print(word)

['1', 'Остап', 'Остап', 'NOUN', '3', 'nsubj']
['2', 'медленно', 'медленно', 'ADV', '3', 'advmod']
['3', 'опустил', 'опускать', 'VERB', '0', 'root']
['4', 'свою', 'свой', 'DET', '5', 'det']
['5', 'руку', 'рука', 'NOUN', '3', 'obj']
['6', 'в', 'в', 'ADP', '9', 'case']
['7', 'изрядно', 'изрядно', 'ADV', '8', 'obl']
['8', 'повреждённый', 'повредить', 'ADJ', '9', 'amod']
['9', 'стул', 'стул', 'NOUN', '3', 'obl']


In [7]:
# /home/fomius2000/Downloads/S+V+INDOBJ.plain.txt

In [11]:
text = ""
with open ("/home/fomius2000/Downloads/S+V+INDOBJ.plain.txt", encoding="utf-8") as f:
    text = str(f.read())
    text = text.split("\n")
    text = tag_ud(text, modelfile=modelfile)
    #checker = pd.DataFrame(checker)
    #checker.head()
    f.close()

In [17]:
print(len(text))
chk = nice_out(len(text))
personolized = []
for i in range(len(text)):
    chk.update()
    root = get_root(text[i])
    place_persons(text[i])
    personolized += [[sout(text[i]).strip(), root]]
for i in range(10):
    print(personolized[i])


352085
99.9% #################################################_['Встревоженный ЧеловекX пошел навстречу мэру.', 'пойти']
['Бывший ЧеловекX превратился в настоятеля.', 'превращаться']
['Старый ЧеловекX прохаживался возле дома.', 'прохаживаться']
['Лопоухий ЧеловекX по-прежнему был в прострации.', 'прострация']
['Высокий ЧеловекX вернулся с граблями;', 'вернуться']
['Сморщенный ЧеловекX сидел за столиком.', 'сидеть']
['Обессилевший ЧеловекX уснул на полу.', 'уснуть']
['Старый Абрахам безмятежно спал за столиком администратора.', 'спать']
['ЧеловекX абсолютно не разбираюсь в музыке.', 'разбираться']
['ЧеловекX абсолютно забыла о своем обещании.', 'забывать']


In [26]:
df = pd.DataFrame(columns=["sentence", "root"])
df["sentence"] = [x[0] for x in personolized]
df["root"]     = [x[1] for x in personolized]
print(df.shape)

df = df.drop(df[df["sentence"].map(len) <= 3].index)
print(df.shape)
df.head()

(352085, 2)
(341043, 3)


,sentence,root,polarity
0,Встревоженный ЧеловекX пошел навстречу мэру.,пойти,0
1,Бывший ЧеловекX превратился в настоятеля.,превращаться,0
2,Старый ЧеловекX прохаживался возле дома.,прохаживаться,0
3,Лопоухий ЧеловекX по-прежнему был в прострации.,прострация,0
4,Высокий ЧеловекX вернулся с граблями;,вернуться,0


In [24]:
df.to_csv("Dataset_persX.csv", index=False)